# Big Shadow State Tomography

In [1]:
import numpy as np

from time import time
from tqdm import tqdm

import qiskit

from qiskit_aer import AerSimulator
from qiskit.visualization import plot_gate_map 

from qiskit_ibm_runtime.fake_provider import FakeLimaV2
from qiskit_ibm_runtime.fake_provider import FakeMontrealV2
from qiskit_ibm_runtime.fake_provider import FakeKyiv

from qiskit.circuit.random import random_circuit

import matplotlib.pyplot as plt

plt.style.use("dark_background")

%cd -q ../..

from rivet_transpiler import transpile_right

In [2]:
FAKE_BACKEND = FakeKyiv()

backend = AerSimulator.from_backend(FAKE_BACKEND)

backend.options.noise_model = None

# qiskit.visualization.plot_gate_map(FAKE_BACKEND)

#### 3.3. Functions

In [3]:
def get_ghz_circuit(qubits_count):

    qubits = list(range(qubits_count))

    circuit = qiskit.QuantumCircuit(qubits_count)

    circuit.h(0)

    for control, target in zip(qubits, qubits[1:]):

        circuit.cx(control, target)

    return circuit

In [4]:
def rotate_qubit_to_basis(circuit, qubit, basis):

    if basis == "X":
        circuit.h(qubit)

    elif basis == "Y":
        circuit.sdg(qubit)
        circuit.h(qubit)

    elif basis == "Z":
        pass

In [5]:
def build_bases_rotation_circuit(bases):

    qubits_count = len(bases)

    circuit = qiskit.QuantumCircuit(qubits_count)

    for qubit, basis in enumerate(bases):

        if basis == "X":
            circuit.h(qubit)

        elif basis == "Y":
            circuit.sdg(qubit)
            circuit.h(qubit)

        elif basis == "Z":
            pass

    return circuit

In [6]:
def get_inverted_channel(channel, qubits_count):

    identity = np.eye(2 ** qubits_count)

    inverted_channel = (2 ** qubits_count + 1) * channel - identity

    return inverted_channel

In [7]:
def get_basis_unitary(basis):

    if basis == "X":
        unitary = 1/np.sqrt(2) * np.array([[1.,1.],
                                           [1.,-1.]])
    elif basis == "Y":
        unitary = 1/np.sqrt(2) * np.array([[1.,-1.0j],
                                           [1.,1.j]])
    elif basis == "Z":
        unitary = np.eye(2)

    return unitary

In [8]:
def get_mean_square_error(a, b):

    mean_square_error = np.abs(((a - b) ** 2).mean())

    return mean_square_error

### Circuit

In [20]:
QUBITS_COUNT = 4
SU2_LAYERS_COUNT = 10
PARAMETRIZATION_RATIO = 0.25

OPTIMIZATION_LEVEL = 3

SEED = 1234

In [21]:
qubits = list(range(QUBITS_COUNT))

rng = np.random.default_rng(SEED) 

#### 4.1.2.  Parametrized SU2 Circuit

In [22]:
# Non-Parametrized Part

left_part = qiskit.QuantumCircuit(QUBITS_COUNT)
right_part = qiskit.QuantumCircuit(QUBITS_COUNT)

non_parametrized_layers_count = SU2_LAYERS_COUNT * 2 * (1 / PARAMETRIZATION_RATIO - 1)

non_parametrized_layers_count = int(non_parametrized_layers_count)

for left_layer in range(non_parametrized_layers_count // 2):
    left_part.rz(0.5, qubits)
    
for right_layer in range(non_parametrized_layers_count // 2):
    right_part.rz(1.5, qubits)   
    
# Parametrized Part

parametrized_part = qiskit.circuit.library.EfficientSU2(
    num_qubits=QUBITS_COUNT,
    reps=SU2_LAYERS_COUNT,
    entanglement="linear",
    skip_final_rotation_layer=True
)

# Compose Parts

parametrized_part = parametrized_part.decompose()

parametrized_circuit = parametrized_part.compose(left_part, front=True)
parametrized_circuit = parametrized_circuit.compose(right_part)

# Bind Parameters

random_parameters = np.random.rand(parametrized_circuit.num_parameters)

circuit = parametrized_circuit.assign_parameters(random_parameters)

print("circuit.depth():", circuit.depth())

circuit.draw(fold=-1)

circuit.depth(): 101


┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────────┐ ┌─────────────┐     ┌─────────────┐┌─────────────┐                              ┌─────────────┐┌─────────────┐                              ┌─────────────┐┌─────────────┐                                ┌─────────────┐┌─────────────┐                              ┌─────────────┐┌─────────────┐                                 ┌─────────────┐┌─────────────┐                              ┌─────────────┐┌─────────────┐                               ┌───────────────┐┌─────────────┐                               ┌─────────────┐┌─────────────┐                              ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐                      
q_0: ┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Ry(0.17732) ├─┤ Rz(0.39207) ├──■──┤ Ry(0.94741) ├┤ Rz(0.92693) ├──────────────────────■───────┤ Ry(0.13955) ├┤ Rz(0.36998) ├──────────────────────■───────┤ Ry(0.53332) ├┤ Rz(0.37388) ├───────────────────────■────────┤ Ry(0.15188) ├┤ Rz(0.29188) ├──────────────────────■───────┤ Ry(0.61855) ├┤ Rz(0.14853) ├────────────────────────■────────┤ Ry(0.76611) ├┤ Rz(0.23838) ├──────────────────────■───────┤ Ry(0.25365) ├┤ Rz(0.95704) ├──────────────────────■────────┤ Ry(0.0041132) ├┤ Rz(0.57869) ├──────────────────────■────────┤ Ry(0.36794) ├┤ Rz(0.67804) ├──────────────────────■───────┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├┤ Rz(1.5) ├──────────────────────
     ├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────────┤ ├─────────────┤┌─┴─┐└─────────────┘├─────────────┤┌─────────────┐     ┌─┴─┐     └─────────────┘└┬────────────┤┌─────────────┐     ┌─┴─┐     └─────────────┘├─────────────┴┐┌─────────────┐     ┌─┴─┐      └─────────────┘├─────────────┤┌─────────────┐     ┌─┴─┐     └─────────────┘├─────────────┤┌──────────────┐      ┌─┴─┐      └─────────────┘└┬────────────┤┌─────────────┐     ┌─┴─┐     └─────────────┘├─────────────┤┌─────────────┐     ┌─┴─┐      └───────────────┘├─────────────┤┌─────────────┐     ┌─┴─┐      └─────────────┘└┬────────────┤┌─────────────┐     ┌─┴─┐     └─────────┘├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤┌─────────┐           
q_1: ┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(0.5) ├┤ Rz(

#### Other Circuits

In [23]:
# ## GHZ Circuit

# circuit = get_ghz_circuit(QUBITS_COUNT)
# circuit.draw(fold=-1)

## Random Circuit

circuit = random_circuit(QUBITS_COUNT, depth=20, max_operands=1, measure=False)
circuit.draw(fold=-1)

┌───┐                 ┌───────────┐    ┌───────────────────┐┌─────┐       ┌───────────┐           ┌───┐               ┌───┐            ┌───┐ ┌────────────┐┌──────┐   ┌────────────┐         ┌────────────┐            ┌───┐        ┌──────────────────┐        ┌───┐                   ┌───┐                  ┌───────────┐       ┌───────────────────────────┐┌──────────────────┐    ┌───┐     
q_0: ────────────┤ Z ├─────────────────┤ P(6.1185) ├────┤ U2(5.8072,3.6359) ├┤ Sdg ├───────┤ P(2.3439) ├───────────┤ S ├───────────────┤ X ├────────────┤ Z ├─┤ Rx(2.9708) ├┤ √Xdg ├───┤ Ry(4.4626) ├─────────┤ U1(1.9426) ├────────────┤ Y ├────────┤ R(3.2181,5.3491) ├────────┤ Z ├───────────────────┤ S ├──────────────────┤ Ry(3.876) ├───────┤ U3(3.4117,1.7607,0.75584) ├┤ R(5.9601,3.6062) ├────┤ T ├─────
                ┌┴───┴┐            ┌───┴───────────┴───┐└───────┬────┬──────┘└┬───┬┘       └──┬──────┬─┘           ├───┤           ┌───┴───┴────┐      ┌┴───┴┐├───────────┬┘└┬───┬─┘┌──┴────────────┴──┐┌─────┴────────────┴────┐    ┌──┴───┴───┐    └──────┬───┬───────┘┌───────┴───┴───────┐┌──────────┴───┴──────────┐      ┌┴───────────┴┐      └───────────┬───┬───────────┘└─────┬─────┬──────┘┌───┴───┴────┐
q_1: ───────────┤ Sdg ├────────────┤ U2(1.4073,4.6232) ├────────┤ √X ├────────┤ T ├───────────┤ √Xdg ├─────────────┤ X ├───────────┤ Ry(5.5782) ├──────┤ Tdg ├┤ P(3.7039) ├──┤ Y ├──┤ R(2.8235,2.0958) ├┤ U(3.34,5.9522,1.3191) ├────┤ P(2.336) ├───────────┤ T ├────────┤ U2(4.7642,4.9717) ├┤ U3(5.4352,6.088,2.5076) ├──────┤ Rz(0.67993) ├──────────────────┤ X ├──────────────────┤ Tdg ├───────┤ Ry(5.1632) ├
         ┌──────┴─────┴──────┐     └┬──────────────────┤        ├───┬┘        ├───┤ ┌─────────┴──────┴────────┐┌───┴───┴────┐┌─────┴────────────┴─────┐└┬───┬┘└───┬───┬───┘ ┌┴───┴┐ └──────┬───┬───────┘└─────────┬───┬─────────┘┌───┴──────────┴───┐       ├───┤        └──────┬─────┬──────┘└──────┬────────────┬─────┘      └───┬──────┬──┘              ┌───┴───┴────┐             └┬───┬┘       └───┬───┬────┘
q_2: ────┤ U2(5.6503,1.3402) ├──────┤ R(4.6402,4.5041) ├────────┤ Z ├─────────┤ I ├─┤ U(1.2096,6.0284,4.1716) ├┤ U1(5.5163) ├┤ U(3.901,4.3772,3.8953) ├─┤ S ├─────┤ I ├─────┤ Sdg ├────────┤ H ├──────────────────┤ S ├──────────┤ R(2.0115,5.8306) ├───────┤ H ├───────────────┤ Tdg ├──────────────┤ U1(4.9993) ├────────────────┤ √Xdg ├─────────────────┤ Rz(2.3907) ├──────────────┤ X ├────────────┤ I ├─────
     ┌───┴───────────────────┴────┐ └─────┬──────┬─────┘ ┌──────┴───┴───────┐ ├───┤ └──────────┬───┬──────────┘└───┬────┬───┘└─────────┬────┬─────────┘ ├───┤     ├───┤     └┬───┬┘        ├───┤           ┌──────┴───┴───────┐  └─┬─────────────┬──┘       ├───┤               └┬───┬┘              └───┬───┬────┘      ┌─────────┴──────┴────────┐        └───┬───┬────┘          ┌───┴───┴────┐       ├───┤     
q_3: ┤ U3(0.87126,0.78459,2.2222) ├───────┤ √Xdg ├───────┤ R(5.7632,3.9823) ├─┤ Y ├────────────┤ S ├───────────────┤ √X ├──────────────┤ √X ├───────────┤ Z ├─────┤ I ├──────┤ S ├─────────┤ Z ├───────────┤ R(5.9297,3.9073) ├────┤ Rz(0.90189) ├──────────┤ H ├────────────────┤ I ├───────────────────┤ I ├───────────┤ U(4.9109,2.3168,2.1053) ├────────────┤ T ├───────────────┤ U1(1.0315) ├───────┤ X ├─────
     └────────────────────────────┘       └──────┘       └──────────────────┘ └───┘            └───┘               └────┘              └────┘           └───┘     └───┘      └───┘         └───┘           └──────────────────┘    └─────────────┘          └───┘                └───┘                   └───┘           └─────────────────────────┘            └───┘               └────────────┘       └───┘

### 4.2.  Run Paulis Experiment

In [24]:
TOTAL_SHOTS_COUNT = 10000

In [25]:
random_bases = rng.choice(['X','Y','Z'], size=(TOTAL_SHOTS_COUNT, QUBITS_COUNT))

bases_list, shots_counts = np.unique(random_bases, axis=0, return_counts=True)

bases_list.shape

(81, 4)

In [26]:
rotation_circuits = []

for bases in bases_list:

    rotation_circuit = build_bases_rotation_circuit(bases)

    rotation_circuits.append(rotation_circuit)

len(rotation_circuits)

81

#### Basic Transpilation - Slower Version!


In [27]:
start_time = time()

# Compose and Transpile all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    measured_circuit = circuit.compose(measurement_circuit)

    # display(measured_circuit.draw(fold=-1))

    shadow_circuit = qiskit.transpile(
        measured_circuit,
        backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)

estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 81/81 [00:04<00:00, 16.85 basis/s]

estimated_time: 4.81


#### Transpile Right - Much Faster!

In [28]:
start_time = time()

# Transpile Central Part

transpiled_central_circuit = qiskit.transpile(
    circuit,
    backend,
    seed_transpiler=SEED,
    optimization_level=OPTIMIZATION_LEVEL)


# Transpile Right all Rotation Circuits

shadow_circuits = []

rotation_tracker = tqdm(rotation_circuits, unit=' basis', ncols=110)

for rotation_circuit in rotation_tracker:

    measurement_circuit = rotation_circuit.measure_all(inplace=False)

    shadow_circuit = transpile_right(
        transpiled_central_circuit,
        measurement_circuit,
        backend=backend,
        seed_transpiler=SEED,
        optimization_level=OPTIMIZATION_LEVEL)

    shadow_circuits.append(shadow_circuit)


estimated_time = time() - start_time

print(f"estimated_time: {estimated_time:.02f}")

100%|█████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 38.50 basis/s]

estimated_time: 2.16
